In [31]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from tscluster.opttscluster import OptTSCluster
from tscluster.preprocessing.utils import load_data

import warnings
warnings.filterwarnings('ignore')

import piccard as pc
import piccard2 as pc2

# unused
import networkx as nx
import math
import re

In [32]:
households_data_2021 = gpd.read_file("data_testing/households_data_2021.geojson")
households_data_2016 = gpd.read_file("data_testing/households_data_2016.geojson")
households_data_2011 = gpd.read_file("data_testing/households_data_2011.geojson")

households_data_2021.rename(columns={'v_CA21_434: Occupied private dwellings by structural type of dwelling data': 'occupied_private_dwellings',
                                     'v_CA21_435: Single-detached house': 'single_detached_house',
                                     'v_CA21_440: Apartment in a building that has five or more storeys': 'apt_five_or_more'}, inplace=True)
households_data_2016.rename(columns={'v_CA16_408: Occupied private dwellings by structural type of dwelling data': 'occupied_private_dwellings',
                                     'v_CA16_409: Single-detached house': 'single_detached_house',
                                     'v_CA16_410: Apartment in a building that has five or more storeys': 'apt_five_or_more'}, inplace=True)
households_data_2011.rename(columns={'v_CA11F_199: Total number of occupied private dwellings by structural type of dwelling': 'occupied_private_dwellings',
                                     'v_CA11F_200: Single-detached house': 'single_detached_house',
                                     'v_CA11F_201: Apartment, building that has five or more storeys': 'apt_five_or_more',}, inplace=True)

In [33]:
census_dfs = [households_data_2011, households_data_2021]
years = ['2011', '2021']

network_table = pc.create_network_table(census_dfs, years, 'GeoUID')
network_table

,geouid_2011,geouid_2021,shape area_2011,quality flags_2011,type_2011,households_2011,pr_uid_2011,dwellings_2011,nhs non-return rate_2011,population_2011,...,region name_2021,area (sq km)_2021,occupied_private_dwellings_2021,single_detached_house_2021,apt_five_or_more_2021,name_2021,dwellings 2016_2021,population 2016_2021,households 2016_2021,network_level_2021
0,2011_5350001.00,2021_5350001.00,6.62211,00000,CT,264,35,283,0.368,604,...,0001.00,6.8192,235.0,15.0,0.0,0001.00,274.0,595.0,247.0,2
1,2011_5350002.00,2021_5350002.00,3.27377,00000,CT,290,35,296,0.307,657,...,0002.00,3.3926,285.0,250.0,0.0,0002.00,279.0,620.0,270.0,2
2,2011_5350003.00,2021_5350003.00,0.94253,02000,CT,227,35,262,NaN,673,...,0003.00,0.9455,265.0,0.0,265.0,0003.00,267.0,749.0,258.0,2
3,2011_5350004.00,2021_5350004.00,0.34347,01000,CT,3416,35,3634,0.391,6781,...,0004.00,0.3404,3275.0,70.0,1790.0,0004.00,3609.0,6686.0,3455.0,2
4,2011_5350005.00,2021_5350005.00,0.37612,01000,CT,2782,35,3472,0.321,5510,...,0005.00,0.3764,3720.0,40.0,2420.0,0005.00,3729.0,6424.0,3424.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,2011_5350820.03,2021_5350820.03,1.15366,00000,CT,1121,35,1134,0.302,4039,...,0820.03,1.1562,1110.0,950.0,0.0,0820.03,1127.0,3888.0,1114.0,2
1223,2011_5350830.00,2021_5350830.00,188.10763,00000,CT,1478,35,1538,NaN,4312,...,0830.00,186.3090,1725.0,1680.0,0.0,0830.00,1705.0,4596.0,1588.0,2
1224,2011_5350831.01,2021_5350831.01,12.20501,00000,CT,2019,35,2058,0.193,5823,...,0831.01,12.1204,2060.0,1650.0,0.0,0831.01,2058.0,5750.0,2031.0,2
1225,2011_5350831.02,2021_5350831.02,17.70258,00000,CT,2475,35,2574,0.230,6510,...,0831.02,17.6159,2805.0,1775.0,135.0,0831.02,2661.0,6780.0,2627.0,2


In [34]:
from tscluster.preprocessing.utils import ntf_to_tnf, tnf_to_ntf
from tscluster.tsplot import tsplot

arr, label_dict = pc2.clustering_prep(network_table, 'name', ['occupied_private_dwellings_2011', 'single_detached_house_2011', 'apt_five_or_more_2011', 'occupied_private_dwellings_2021', 'single_detached_house_2021', 'apt_five_or_more_2021'])
fig, ax = tsplot.plot(X=arr, label_dict=label_dict)

AttributeError: 'list' object has no attribute 'shape'

In [26]:
# we will use the elbow method to determine the optimal number of clusters
import setuptools
from sympy import false
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

model = KMeans(random_state=4,n_init=10)
visualizer = KElbowVisualizer(model, k=(2,10),timings=False, ax=plt.gca())
visualizer.fit(arr)       # Fit data to visualizer
plt.title('Elbow Method for Optimal k using Sum of Square Error')
plt.xlabel('k')
plt.ylabel('Sum of Squared Error')
plt.show()

ModuleNotFoundError: No module named 'setuptools'

In [27]:
# Initialize the model
opt_ts = OptTSCluster(
    n_clusters=3,
    scheme='z1c1',
    n_allow_assignment_change=None # Allow as many changes as possible
)

arr = np.nan_to_num(arr, nan = -1)

# Assign clusters
opt_ts.fit(arr, label_dict=label_dict)

Warm starting...
Done with warm start after 0.03secs

Set parameter Username
Set parameter LicenseID to value 2673747
Academic license - for non-commercial use only - expires 2026-05-30
Obj val: [4075.]

Total time is 327.73secs



In [28]:
cluster_label_fdc = opt_ts.labels_
# the cluster labels for the first 5 entities
cluster_label_fdc[:10]

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [2, 0],
       [0, 1]])

In [29]:
fig, ax = tsplot.plot(
    X=arr, # the temporal data
    cluster_centers=opt_ts.cluster_centers_, # the cluster centers
    labels=opt_ts.labels_, label_dict=label_dict, # the cluster labels and the label dictionary
    entity_idx=[label_dict['N'].index(i) for i in opt_ts.get_dynamic_entities()[0] if i % 100 == 0], # show every 100 entities that exhibit cluster label changes
    show_all_entities=False,figsize=(7,16), annot_fontsize = 10,
    xlabel='Year',ylabel='Number'
    ) # show only the entities that exhibit cluster label changes

ValueError: Shape of passed values is (1227, 2), indices imply (1226, 2)

In [ ]:
import seaborn as sns
from typing import List

def plot_cluster_heatmap(X: np.ndarray,
                         cluster_result: OptTSCluster,
                         n_clusters:int,
                         cluster_labels: List[str]|None = None,
                         label_dict: dict|None = None,
                         figsize: tuple = (10, 7)) -> plt.Figure:
    """
    Plot the cluster heatmap for the temporal data based on the cluster result.

    Parameters
    ----------
    temporal_data : np.ndarray
        The temporal data with shape (n_timesteps, n_samples, n_features).
    cluster_result : OptTSCluster|TSGlobalKmeans|KMeans
        The cluster result from three clustering method of *tscluster*
    n_clusters : int
        The number of clusters in the cluster result.
    cluster_labels : List[str], optional
        The cluster labels, by default None.
        If None, the cluster labels will be 'Cluster 1', 'Cluster 2', ...
    label_dict : dict, optional
        The label dictionary of the temporal data, by default None
        If None, the label dictionary of the cluster result will be used.

    figsize : tuple, optional
        The figure size of the heatmap, by default (10, 7)

    Returns
    -------
    plt.Figure
        The heatmap figure.
    """
    if label_dict is None:
        label_dict = cluster_result.label_dict_

    z_score_matrix = [[[] for _ in range(X.shape[2])]
                      for _ in range(n_clusters)]
    for fsa in range(X.shape[1]):
        current_fsa = cluster_result.labels_[fsa]
        for year in range(X.shape[0]):
            for feature in range(X.shape[2]):
                z_score_matrix[current_fsa[year]][feature].append(X[year,fsa,feature])
    z_score_matrix = np.array(z_score_matrix, dtype=object).T
    z_score_matrix = np.array([[np.mean(z_score_matrix[feature][cluster])
                                for cluster in range(n_clusters)]
                               for feature in range(X.shape[2])])
    result = pd.DataFrame(z_score_matrix, index=label_dict['F'])
    result = result.astype(float)
    if cluster_labels is not None:
        result.columns = cluster_labels
    else:
        result.columns = ['Cluster ' + str(i+1) for i in range(cluster_result.n_clusters)]

    plt.figure(figsize=figsize)
    fig = sns.heatmap(result, cmap='coolwarm', annot=True, fmt=".2f")
    return plt.gcf()

fig = plot_cluster_heatmap(X = arr,
                           cluster_result=opt_ts,
                           n_clusters=3,
                           cluster_labels=['c1','c2','c3'])
                           # assign the cluster labels
plt.title('Heatmap of features by clusters')
plt.xlabel('Clusters')
plt.ylabel('Features')
plt.show()